In [ ]:
# 1 项目背景介绍：

In [ ]:
# 本分析以支付宝营销活动为例，通过广告点击率指标比较两组营销策略的广告投放效果。

In [1]:
# 2 数据预处理:

In [3]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline 

In [4]:
# 读取数据
data = pd.read_csv('./effect_tb.csv', header=None)
data.columns = ["dt","user_id","label","dmp_id"]

In [5]:
# 删除不相关的“天数”属性一列:
data = data.drop(columns= "dt")
data.head(3)

,user_id,label,dmp_id
0,1,0,1
1,1000004,0,1
2,1000004,0,2


In [6]:
#查看表格数据一般描述：

data.describe()

,user_id,label,dmp_id
count,2.645958e+06,2.645958e+06,2.645958e+06
mean,3.112995e+06,1.456297e-02,1.395761e+00
std,1.828262e+06,1.197952e-01,6.920480e-01
min,1.000000e+00,0.000000e+00,1.000000e+00
25%,1.526772e+06,0.000000e+00,1.000000e+00
50%,3.062184e+06,0.000000e+00,1.000000e+00
75%,4.721132e+06,0.000000e+00,2.000000e+00
max,6.265402e+06,1.000000e+00,3.000000e+00


In [7]:
# 重复值检验 & 删除重复值：
data.shape

(2645958, 3)

In [8]:
data.nunique()

user_id    2410683
label            2
dmp_id           3
dtype: int64

In [9]:
data[data.duplicated(keep=False)].sort_values(by = ["user_id"])

,user_id,label,dmp_id
8529,1027,0,1
1485546,1027,0,1
1579415,1471,0,1
127827,1471,0,1
404862,2468,0,1
...,...,...,...
1382121,6264633,0,1
1382245,6264940,0,1
2575140,6264940,0,1
1382306,6265082,0,3


In [10]:
data = data.drop_duplicates()

In [11]:
data[data.duplicated(keep=False)]   
    # 确认没有任何其他的重复值。

,user_id,label,dmp_id


In [14]:
# 确认空缺值数据：
data.info(show_counts= True)       
    #数据并无空值，无需进行处理。

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2632975 entries, 0 to 2645957
Data columns (total 3 columns):
 #   Column   Non-Null Count    Dtype
---  ------   --------------    -----
 0   user_id  2632975 non-null  int64
 1   label    2632975 non-null  int64
 2   dmp_id   2632975 non-null  int64
dtypes: int64(3)
memory usage: 144.9 MB


In [15]:
# 异常值检测：
data.pivot_table(index= "dmp_id",columns="label",values= "user_id",aggfunc= "count",margins=True)
    # 字段并未发现异常值，无需进行处理。

label,0,1,All
dmp_id,,,
1,1881745,23918,1905663
2,404811,6296,411107
3,307923,8282,316205
All,2594479,38496,2632975


In [16]:
# 数据类型检测：
data.dtypes
    #数据类型正常，无需处理。

user_id    int64
label      int64
dmp_id     int64
dtype: object

In [18]:
# 样本容量检测：
# a)首先以对照组的点击率设置为基准线:
data[data["dmp_id"]== 1]["label"].mean()

0.012551012429794775

In [19]:
#得到的对照组点击率为1.26%，假设我们希望新的营销策略可以让广告点击率提升至少！%的话，
#那么接下来我们可以通过输入点击率基准线&最小提升比例来得到我们想要的最小样本量。
#这里用到的测算最小样本量的工具为：Evan Miller样本量计算工具。
#具体链接如下：https://www.evanmiller.org/ab-testing/sample-size.html
#那么通过工具我们得知，最小样本量应该为：2167

In [20]:
# 验证营销活动的样本量是否满足最小样本量的条件：
data["dmp_id"].value_counts()
    #两场营销活动的样本量分别为41.11万和31.62万。满足最小样本量的条件。

1    1905663
2     411107
3     316205
Name: dmp_id, dtype: int64

In [21]:
data.to_csv("./output.csv", index= False)

In [22]:
data= pd.read_csv("./output.csv")

In [23]:
# 3 假设检验

In [24]:
# 3.1 观察几组实验的点击率情况：
print("对照组：",data[data["dmp_id"]==1]["label"].mean())
print("营销策略组一：", data[data["dmp_id"]== 2]["label"].mean())
print("营销策略组二：", data[data["dmp_id"]== 3]["label"].mean())

对照组： 0.012551012429794775
营销策略组一： 0.015314747742072015
营销策略组二： 0.026191869198779274


In [25]:
# 总体来说，策略一和策略二的点击率相对于对照组都有所提升。
# 不同的是，策略一提升了约0.2%；而策略二提升了约1.3%。
# 因此，策略二满足了前边我们提到的对于“点击率提升最小值”的要求。
# 接下来需要进行假设检验，看策略二点击率的提升是否显著。

In [26]:
# 3.2:假设检验：

In [27]:
# 3.2.1 零假设和备择假设：
#记对照组点击率为 p1，策略二点击率为 p2，则：
#零假设 H0： p1 ≥ p2
#备择假设 H1： p1 ＜ p2

In [28]:
# 3.2.2 检验类型&显著性水平：
#样本大小n> 30,总体均值和标准差未知，所以采用 Z检验。显著性水平α取 0.05。

In [29]:
# 3.2.3 计算用户数，点击数，点击率，总和点击率：

In [30]:
# 用户数：
n_old = len(data[data.dmp_id ==1])  #对照组
n_new = len(data[data.dmp_id ==3])  #策略二

In [38]:
# 点击数：
c_old = len(data[data.dmp_id ==1][data.label ==1])
c_new = len(data[data.dmp_id ==3][data.label ==1])

<ipython-input-38-f98909f9b1e1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c_old = len(data[data.dmp_id ==1][data.label ==1])
<ipython-input-38-f98909f9b1e1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c_new = len(data[data.dmp_id ==3][data.label ==1])


In [32]:
# 计算点击率：
r_old = c_old/n_old
r_new = c_new/n_new

In [33]:
# 总和点击率：
r = (c_old+c_new)/(n_old+n_new)

In [34]:
print("总和点击率：",r)

总和点击率： 0.014492310074225832


In [36]:
#3.2.4 计算检验统计量Z:
z= (r_old-r_new)/ np.sqrt(r*(1-r)*(1/n_old + 1/n_new))
print ("检验统计量z:", z)

检验统计量z: -59.44168632985996


In [37]:
# a=0.05是对应的z分位数为：
from  scipy.stats import norm
z_alpha = norm.ppf(0.05) 
z_alpha

-1.6448536269514729

In [39]:
# z_alpha = -1.64,检验统计量z = -59.44。
#该检验为左侧单尾检验，拒绝域为{z＜z_alpha}。
# 因此，我们可以得出的结论是：原假设不成立，策略二点击率的提升在统计上是显著的。

In [ ]:
# 4 结论：
# 通过上述的ab实验计算，我们可以得知：
# 两种营销策略中，只有策略二是对广告点击率有显著提升效果的。相较于对照组点击率提升了近一倍。（1.25% to 2.61%）
# 因而在两组营销策略中应该选择第二种进行推广。